# 练习1

Q: 如果你有一个数百万特征的训练集，你应该选择哪种线性回归训练算法？

A: 随机梯度下降或者小批量梯度下降

# 练习2

Q: 假设你的训练集中特征的数据尺度（scale）有着非常大的差异，哪种算法会受到影响？有多大的影响？对于这些影响你可以做什么？

A: 梯度下降算法会受到影响。速度可能会变得很慢。在使用梯度下降之前，首先将数据约束到有相近的尺度范围。

# 练习3

Q: 训练Logistic回归模型时，梯度下降是否会陷入局部最低点？

A: 不会。因为Logistic回归模型的损失函数是一个凸函数。

# 练习4

Q: 在有足够的训练时间下，是否所有的梯度下降都会得到相同的模型参数？

A: 不会。因为随机梯度下降和小批量梯度下降损失值最终会在局部最小值附近震荡，而不是确定的一个值。所以模型参数不会都相同，但是很接近。

# 练习5

Q: 假设你使用批量梯度下降法，画出每一代的验证误差。当你发现验证误差一直增大，接下来会发生什么？你怎么解决这个问题？

A: 接下来误差会越来越大。应该尝试：减小学习率。

# 练习6

Q: 当验证误差升高时，立即停止小批量梯度下降是否是一个好主意？

A: 不是的。因为小批量提取下降误差变化的曲线不是平滑的，会有局部的震荡。一个好的办法是：当一段时间后，验证误差一直在升高或变化很小时，停止训练。

# 练习7

Q: 哪个梯度下降算法（我们讨论的那些算法中）可以最快到达解的附近？哪个的确实会收敛？怎么使其他算法也收敛？

A: 小批量梯度下降算法会最快到达解的附近。批量梯度下降会收敛。要使得别的算法也收敛，可以根据动态的减小学习率。

# 练习8

Q: 假设你使用多项式回归，画出学习曲线，在图上发现学习误差和验证误差之间有着很大的间隙。这表示发生了什么？有哪三种方法可以解决这个问题？

A: 发生了过拟合。解决办法为：1. 搜集更多的数据扩充训练集；2. 对模型使用正则化；3. 使用更加简单的模型。

# 练习9

Q: 假设你使用岭回归，并发现训练误差和验证误差都很高，并且几乎相等。你的模型表现是高偏差还是高方差？这时你应该增大正则化参数$\alpha$，还是降低它？

A: 高偏差，也就是欠拟合。应该减小正则化参数，这样会提高模型的自由度，让模型能够更好的拟合数据。

# 练习10

Q: 你为什么要这样做：
    - 使用岭回归代替线性回归？
    - Lasso回归代替岭回归？
    - 弹性网络代替Lasso回归？

A: 那就告诉你呗：
    - 模型过拟合，需要使用正则化减小模型自由度
    - 模型只有少数参数对预测有效
    - 训练集的特征数量较大时，Lasso可能出现不规律的情况

# 练习11

Q: 假设你想判断一幅图片是室内还是室外，白天还是晚上。你应该选择二个逻辑回归分类器还是一个Softmax分类器？

A: 二个逻辑回归分类器。因为一个Softmax尽管能够针对多类别进行输出，但是一次只能输出一个结果。

# 练习12

Q: 在Softmax回归上应用批量梯度下降的早期停止法（不使用Scikit-Learn）。

A: 见以下代码

使用sklearn获取数据(iris)，并进行train-test分割

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

首先加载iris数据

In [2]:
iris = datasets.load_iris()
X = iris['data']
y = iris['target']

在多类别分类中，我们需要把标签转换成one-hot编码

In [3]:
def one_hot(y):
    n_classes = len(set(y))
    m = len(y)
    y_one_hot = np.zeros((m, n_classes))
    y_one_hot[range(m), y] = 1
    return y_one_hot

In [4]:
one_hot([0, 1, 2])

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

看起来不错

接下来，我们把标签转换成one-hot编码，并且分割数据集

In [5]:
y = one_hot(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X shape:", X_train.shape)
print("y shape:", y_train.shape)

X shape: (120, 4)
y shape: (120, 3)


定义一些函数，包括softmax, predict, error_function, gradient

In [6]:
def softmax(X):
    X = np.array(X)
    return np.exp(X) / np.exp(X).sum(axis=1).reshape((X.shape[0], 1))

In [7]:
softmax([[1, 2, 3], [2, 3, 6]])

array([[0.09003057, 0.24472847, 0.66524096],
       [0.01714783, 0.04661262, 0.93623955]])

In [8]:
def predict(X, theta):
    X_bias = np.c_[np.ones((X.shape[0], 1)), X]
    z = X_bias.dot(theta.T)
    return softmax(z)


def error_log_entropy(X, y, theta):
    y_predict = predict(X, theta)
    n_classes = y.max() + 1
    m = X.shape[0]
    error = -1. / m * (np.log(y_predict) * y).sum().sum()
    return error


def get_gradient(X, y, y_pred, theta):
    m = X.shape[0]
    X_bias = np.c_[np.ones((X.shape[0], 1)), X]
    return 1. / m * ((y_pred - y).T.dot(X_bias))

In [9]:
predict(np.array([[1, 2, 3], [3, 4, 4]]), np.array([[0, 1, 1, 1], [0, 1, 1, 1], [0, 1, 1, 1]]))

array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]])

In [10]:
error_log_entropy(X_train, y_train, np.array([[0, 1, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 1, 1]]))

1.0986122886681098

In [11]:
theta = np.array([[0, 1, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 1, 1]])
get_gradient(X, y, predict(X, theta), theta)

array([[ 2.12792746e-16,  2.79111111e-01, -1.21333333e-01,
         7.64888889e-01,  3.18222222e-01],
       [-1.20274161e-16, -3.08888889e-02,  9.46666667e-02,
        -1.67111111e-01, -4.24444444e-02],
       [-4.11522668e-16, -2.48222222e-01,  2.66666667e-02,
        -5.97777778e-01, -2.75777778e-01]])

接下来，开始我们的迭代

In [12]:
n_iterations = 1000
eta = 0.1
m, n = X_train.shape
n_classes = y_train.shape[1]
theta = np.concatenate([np.c_[np.ones((1, 1)), np.random.random((1, n))] for _ in range(n_classes)])
best_loss = float("inf")
best_iter = 0
best_theta = theta
loss_time = 0
MAX_LOSS_TIME = 5
for i in range(n_iterations):
    loss = error_log_entropy(X_train, y_train, theta)
    y_pred = predict(X, theta)
    gradient = get_gradient(X, y, y_pred, theta)
    theta = theta - eta * gradient
    loss_delta = best_loss - loss
    if 0 < loss_delta < 0.0001:
        loss_time += 1
    if loss_time == MAX_LOSS_TIME:
        break
    if loss < best_loss:
        best_loss = loss
        best_iter = i
        best_theta = theta

In [13]:
print("Best loss:", best_loss)
print("Best iter:", best_iter)
print("Best theta:", best_theta)

Best loss: 0.14990847846651215
Best iter: 690
Best theta: [[ 1.43986895  1.05252648  2.66766967 -2.00019001 -0.48141481]
 [ 1.49506949  1.19777522  0.27683077  0.29298624 -0.33871456]
 [ 0.06506156 -0.91480279 -0.94119927  2.97148979  2.71010066]]


In [14]:
def accuray(X, y):
    y_pred = np.argmax(predict(X, best_theta), axis=1)
    y = np.argmax(y, axis=1)
    return (y_pred == y).sum() / y.shape[0]

In [15]:
print("Training Set:", accuray(X_train, y_train))
print("Test Set:", accuray(X_test, y_test))

Training Set: 0.975
Test Set: 1.0


# TODO

1. 涉及公式的推导
2. 公式到代码快速转换的能力的培养